In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    # Tenta usar DefaultAzureCredential
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Se falhar, usa InteractiveBrowserCredential
    credential = InteractiveBrowserCredential()

# Conectar ao workspace do Azure ML
ml_client = MLClient.from_config(credential=credential)



Exception in thread Thread-6 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\pablo\anaconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\pablo\anaconda3\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\pablo\anaconda3\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc6 in position 8: invalid continuation byte
DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no respons

In [2]:
import os

# Criar a pasta 'src' caso não exista
os.makedirs("src", exist_ok=True)


In [3]:
%%writefile src/sorvetes-training.py

# Importação das bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

# Carregar o dataset
print("Carregando os dados...")
sorvetes = pd.read_csv('venda_sorvete.csv')

# Separar features e labels
X = sorvetes[['Temperatura (°C)']].values
y = sorvetes['Quantidade Vendida'].values

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Definir hiperparâmetro de regularização
reg = 0.01

# Treinar um modelo de regressão logística
print("Treinando modelo com taxa de regularização de", reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calcular a acurácia
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print("Acurácia:", acc)

# Calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:, 1])
print("AUC:", str(auc))


Overwriting src/sorvetes-training.py


In [5]:
from azure.ai.ml import command
from azure.ai.ml.entities import Environment

# Definir um ambiente do Azure ML
custom_env = Environment(
    name="meu-ambiente-ml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest",
)

# Criar o job com o ambiente especificado
job = command(
    code="./src",  # Caminho para o código-fonte
    command="python sorvetes-training.py",  # Comando para executar o script
    compute="cpu-cluster",  # Nome do cluster
    environment=custom_env,  # Definir o ambiente
    experiment_name="sorvetes-training",  # Nome do experimento
)

# Submeter o job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)


Monitor your job at https://ml.azure.com/runs/loyal_hand_8m33qpl58h?wsid=/subscriptions/bffb6fbc-dd6a-46a8-baf8-3023c79a9322/resourcegroups/rg-dio-projeto-1/workspaces/workspace-dio&tid=70dc6ec4-74b1-4c27-81f9-d02ee2e57396
